In [1]:
import pandas as pd
import numpy as np
import re
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

In [12]:
data = pd.read_excel('人工智能私募融资.xlsx', sheet_name='赛迪排名')
year = pd.read_excel('人工智能私募融资.xlsx', sheet_name='年份投资')
cycle = pd.read_excel('人工智能私募融资.xlsx',sheet_name='轮次')
df_t=pd.DataFrame(data[['公司','技术']])
df_p=pd.DataFrame(data[['公司','应用行业']])
df_a=pd.DataFrame(data[['公司','地域']])
df_u=pd.DataFrame(data[['公司','高校']])
df_u['高校'].fillna('',inplace=True)
df_c=pd.DataFrame(data[['公司','依托龙头']])

In [6]:
# 获取所有类别
all_t=[]
for x in df_t.技术:
    all_t.extend(x.split('，'))
tech=pd.unique(all_t)
# 生成默认全为0的矩阵
zero_matrix = np.zeros((len(data), len(tech)))
dummies_t=pd.DataFrame(zero_matrix, columns=tech)
# 按行赋值生成虚拟变量矩阵
for i,gen in enumerate(df_t.技术):
    indices=dummies_t.columns.get_indexer(gen.split('，'))
    dummies_t.iloc[i, indices] = 1
# 数据展示
data_tech = df_t.join(dummies_t)
data_tech

,公司,技术,计算机视觉,数据挖掘/云平台,机器学习,自然语言处理,智能语音技术,生物识别,机器人技术,知识图谱
0,字节跳动,计算机视觉，数据挖掘/云平台，机器学习，自然语言处理,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
1,华大基因,计算机视觉，机器学习,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,京东数科,计算机视觉，数据挖掘/云平台，智能语音技术，机器学习，自然语言处理，生物识别,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
3,大疆创新,计算机视觉，数据挖掘/云平台，机器学习，机器人技术,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
4,小米,计算机视觉，数据挖掘/云平台，智能语音技术，机器学习，自然语言处理，知识图谱，生物识别,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0
5,浪潮云,数据挖掘/云平台，知识图谱,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
6,商汤科技,计算机视觉，机器学习，生物识别,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
7,寒武纪科技,机器学习,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
8,旷视科技,计算机视觉，数据挖掘/云平台，机器学习，生物识别,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
9,思必驰,数据挖掘/云平台，智能语音技术，自然语言处理,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0


In [7]:
# 获取所有类别
all_p=[]
for x in df_p.应用行业:
    all_p.extend(x.split('，'))
prof=pd.unique(all_p)
# 生成默认全为0的矩阵
zero_matrix = np.zeros((len(data), len(prof)))
dummies_p=pd.DataFrame(zero_matrix, columns=prof)
# 按行赋值生成虚拟变量矩阵
for i,gen in enumerate(df_p.应用行业):
    indices=dummies_p.columns.get_indexer(gen.split('，'))
    dummies_p.iloc[i, indices] = 1
# 数据展示
data_prof = df_p.join(dummies_p)
data_prof

,公司,应用行业,文娱,医疗健康,金融,工业制造,移动终端,营销,安防,零售,智慧出行,基础元件,客服,教育,农业
0,字节跳动,文娱,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,华大基因,医疗健康,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,京东数科,金融,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,大疆创新,工业制造,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,小米,移动终端,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,浪潮云,移动终端，营销,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,商汤科技,安防，零售，智慧出行，移动终端,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
7,寒武纪科技,基础元件,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
8,旷视科技,安防，零售,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
9,思必驰,智慧出行，移动终端，客服,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0


In [8]:
def classify(uni):
    uni = re.sub('北大', 'C9', uni)
    uni = re.sub('清华', 'C9', uni)
    uni = re.sub('复旦', 'C9', uni)
    uni = re.sub('上交', 'C9', uni)
    uni = re.sub('南大', 'C9', uni)
    uni = re.sub('浙大', 'C9', uni)
    uni = re.sub('中科大', 'C9', uni)
    uni = re.sub('哈工大', 'C9', uni)
    uni = re.sub('港大', '非内地', uni)
    uni = re.sub('哈佛', '非内地', uni)
    uni = re.sub('斯坦福', '非内地', uni)
    uni = re.sub('耶鲁', '非内地', uni)
    uni = re.sub('佐治亚理工', '非内地', uni)
    uni = re.sub('麻省', '非内地', uni)
    uni = re.sub('剑桥', '非内地', uni)
    uni = re.sub('南开', '优985', uni)
    uni = re.sub('武大', '优985', uni)
    uni = re.sub('同济', '优985', uni)
    uni = re.sub('华科', '优985', uni)
    uni = re.sub('国科大', '优985', uni)
    uni = re.sub('北航', '优985', uni)
    uni = re.sub('厦大', '优985', uni)
    uni = re.sub('人大', '优985', uni)
    uni = re.sub('华工', '优985', uni)
    uni = re.sub('中山', '优985', uni)
    uni = re.sub('电子科大', '优985', uni)
    uni = re.sub('中科院', '非985', uni)
    uni = re.sub('上财', '非985', uni)
    return uni

In [9]:
#将大学划分为四类
for i in range(len(df_u.高校)):
    if len(df_u.高校[i]) !=0  :
        df_u.高校[i]=classify(df_u.高校[i])
# 获取所有类别
all_u=[]
for x in df_u.高校:
    if x=='':
        continue
    all_u.extend(x.split('，'))
univ=pd.unique(all_u)
# 生成默认全为0的矩阵
zero_matrix = np.zeros((len(data), len(univ)))
dummies_u=pd.DataFrame(zero_matrix, columns=univ)
# 按行赋值生成虚拟变量矩阵
for i,gen in enumerate(df_u.高校):
    indices=dummies_u.columns.get_indexer(gen.split('，'))
    c=Counter(indices)
    k = c.most_common(len(c))
    for j in k:
        dummies_u.iloc[i, j[0]] = j[1]
# 数据展示
data_univ = df_u.join(dummies_u)
data_univ

,公司,高校,C9,优985,无高校依托,非内地,非985
0,字节跳动,C9，优985,1.0,1.0,0.0,0.0,0.0
1,华大基因,无高校依托,0.0,0.0,1.0,0.0,0.0
2,京东数科,C9,1.0,0.0,0.0,0.0,0.0
3,大疆创新,非内地,0.0,0.0,0.0,1.0,0.0
4,小米,优985,0.0,1.0,0.0,0.0,0.0
5,浪潮云,无高校依托,0.0,0.0,1.0,0.0,0.0
6,商汤科技,非内地，C9，C9,2.0,0.0,0.0,1.0,0.0
7,寒武纪科技,无高校依托,0.0,0.0,1.0,0.0,0.0
8,旷视科技,C9,1.0,0.0,0.0,0.0,0.0
9,思必驰,C9，C9，非内地,2.0,0.0,0.0,1.0,0.0


In [10]:
# 获取所有类别
all_a=[]
for x in df_a.地域:
    all_a.extend(x.split('，'))
area=pd.unique(all_a)
# 生成默认全为0的矩阵
zero_matrix = np.zeros((len(data), len(area)))
dummies_a=pd.DataFrame(zero_matrix, columns=area)
# 按行赋值生成虚拟变量矩阵
for i,gen in enumerate(df_a.地域):
    indices=dummies_a.columns.get_indexer(gen.split('，'))
    dummies_a.iloc[i, indices] = 1
# 数据展示
data_area = df_a.join(dummies_a)
data_area

,公司,地域,京,粤,沪,其他
0,字节跳动,京,1.0,0.0,0.0,0.0
1,华大基因,粤,0.0,1.0,0.0,0.0
2,京东数科,京,1.0,0.0,0.0,0.0
3,大疆创新,粤,0.0,1.0,0.0,0.0
4,小米,京,1.0,0.0,0.0,0.0
5,浪潮云,京,1.0,0.0,0.0,0.0
6,商汤科技,京,1.0,0.0,0.0,0.0
7,寒武纪科技,粤,0.0,1.0,0.0,0.0
8,旷视科技,京,1.0,0.0,0.0,0.0
9,思必驰,沪,0.0,0.0,1.0,0.0


In [13]:
# 获取所有类别
all_c=[]
for x in df_c.依托龙头:
    all_c.extend(x.split('，'))
comp=pd.unique(all_c)
# 生成默认全为0的矩阵
zero_matrix = np.zeros((len(data), len(comp)))
dummies_c=pd.DataFrame(zero_matrix, columns=comp)
# 按行赋值生成虚拟变量矩阵
for i,gen in enumerate(df_c.依托龙头):
    indices=dummies_c.columns.get_indexer(gen.split('，'))
    dummies_c.iloc[i, indices] = 1
# 数据展示
data_comp = df_c.join(dummies_c)
data_comp

,公司,依托龙头,无,京东,阿里,腾讯,百度,盛大,华为
0,字节跳动,无,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,华大基因,无,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,京东数科,京东,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,大疆创新,无,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,小米,无,1.0,0.0,0.0,0.0,0.0,0.0,0.0
5,浪潮云,无,1.0,0.0,0.0,0.0,0.0,0.0,0.0
6,商汤科技,阿里,0.0,0.0,1.0,0.0,0.0,0.0,0.0
7,寒武纪科技,阿里,0.0,0.0,1.0,0.0,0.0,0.0,0.0
8,旷视科技,阿里,0.0,0.0,1.0,0.0,0.0,0.0,0.0
9,思必驰,阿里,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [15]:
data1=dummies_t.join(data[['私募投资规模']])
data2=dummies_p.join(data[['私募投资规模']])
data3=dummies_u.join(data[['私募投资规模']])
data4=dummies_a.join(data[['融资时间','私募投资规模']])
data5=dummies_c.join(data[['私募投资规模']])
data6=data[['融资时间','私募投资规模']]
data_reg=dummies_t.join(dummies_p.join(dummies_u.join(dummies_a.join(data[['融资时间','私募投资规模']]))))
# clas=data_reg.columns.tolist()
# data_reg
# print(data_reg.corr())
def regre(datan):
    X = datan.iloc[:, :-1].values
    Y = datan.iloc[:, -1].values

    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.15)
    print("原始数据特征:", datan.iloc[:, :-1].shape,
          ",训练数据特征:", X_train.shape,
          ",测试数据特征:", X_test.shape)

    print("原始数据标签:", datan.iloc[:, -1].shape,
          ",训练数据标签:", Y_train.shape,
          ",测试数据标签:", Y_test.shape)
#     ss_x = StandardScaler()
#     X_train = ss_x.fit_transform(X_train)
#     X_test = ss_x.transform(X_test)
#     ss_y = StandardScaler()
#     Y_train = ss_y.fit_transform(Y_train.reshape(-1, 1))
#     Y_test = ss_y.transform(Y_test.reshape(-1, 1))
    
    rfr = RandomForestRegressor()
    rfr.fit(X_train, Y_train)
    rfr_pred = rfr.predict(X_test)

    etr = ExtraTreesRegressor()
    etr.fit(X_train, Y_train)
    etr_pred = etr.predict(X_test)

    gbr = GradientBoostingRegressor()
    gbr.fit(X_train, Y_train)
    gbr_pred = gbr.predict(X_test)

    print('随机森林回归', '默认评估值：', rfr.score(X_test, Y_test), 'R_squared:', r2_score(Y_test, rfr_pred),'极端森林回归', '默认评估值：', etr.score(X_test, Y_test), 'R_squared:', r2_score(Y_test, etr_pred),'梯度提升回归', '默认评估值：', gbr.score(X_test, Y_test), 'R_squared:', r2_score(Y_test, gbr_pred))
    regressor = LinearRegression()
    regressor=regressor.fit(X_train, Y_train)
    a = regressor.intercept_
    b = regressor.coef_
    print('最佳拟合截距：',a,',回归系数：',b)
    score = regressor.score(X_test,Y_test)
    print('R方:',score)
    Y_pred = regressor.predict(X_test)
    print('预测值:',Y_pred)
#     plt.plot(range(len(Y_pred)),Y_pred,'b',label="predict")
#     plt.plot(range(len(Y_pred)),Y_test,'r',label="true")

# # plt.scatter(X_test, Y_test, color='red')
# # plt.scatter(X_test, Y_pred, color='blue')

# #显示图像
#     plt.show()

In [16]:
f=pd.ExcelWriter('regression.xlsx')
data1.to_excel(f,sheet_name='技术')
data2.to_excel(f,sheet_name='行业')
data3.to_excel(f,sheet_name='高校')
data4.to_excel(f,sheet_name='地域')
data5.to_excel(f,sheet_name='依托龙头')
data6.to_excel(f,sheet_name='时间')
data_reg.to_excel(f,sheet_name='总')
f.save()

In [166]:
regre(data3)
# 回归效果太差，因为类别太多，各因素间关联性不强，偶然性较大
# 改为选用频数分析，相关性分析，热力图分析，尚且未完成

原始数据特征: (112, 5) ,训练数据特征: (95, 5) ,测试数据特征: (17, 5)
原始数据标签: (112,) ,训练数据标签: (95,) ,测试数据标签: (17,)
随机森林回归 默认评估值： -2.8202774586093566 R_squared: -2.8202774586093566 极端森林回归 默认评估值： -4.176268419067047 R_squared: -4.176268419067047 梯度提升回归 默认评估值： -6.473712772941707 R_squared: -6.473712772941707
最佳拟合截距： -33619.42771970868 ,回归系数： [157029.89830794 222546.04177201 265150.42438021  81997.94556809
 123678.42075771]
R方: -4.835442711684186
预测值: [411472.65582431 123410.47058824 634018.69759632 123410.47058824
  90058.993038   123410.47058824 188926.6140523  617149.02215011
 188926.6140523  123410.47058824 123410.47058824 394602.9803781
  90058.993038   188926.6140523   90058.993038   231530.9966605
 188926.6140523 ]


C:\Users\Kaiqi Zhang\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Kaiqi Zhang\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [88]:
# VIF 多重共线性

,计算机视觉,机器学习,数据挖掘/云平台,自然语言处理,智能语音技术,生物识别,机器人技术,知识图谱,医疗健康,文娱,...,C9,优985,非内地,非985,粤,京,沪,其他,融资时间,私募投资规模
0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,6,399580
1,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,8,4793030
2,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,4,1965000
3,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,7,799720
4,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,9,1024300
5,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2,80000
6,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,2.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,6,1973580
7,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3,367000
8,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,9,874238
9,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,8,81300
